# Proyectos de Infraestructura Vial - MinObras

El Ministerio de Transporte (MinTransporte) cuenta con un presupuesto de **100 millones de pesos** para ejecutar obras de infraestructura vial, con el fin de generar desarrollo en el país.

MinTransporte le va a proporcionar datos que contiene información sobre:

- **Costo de ejecución** de cada proyecto (en millones de pesos)
- **Número de empleos generados** (en miles)
- **Ubicación geográfica** del proyecto (latitud y longitud)

El objetivo del Ministerio es **maximizar la cantidad total de empleos generados**, respetando el límite presupuestal.

## Datos

Generamos un conjunto artificial de datos para simular un portafolio de obras públicas, distribuidas geográficamente en cinco ciudades de Colombia:

- **Bogotá**
- **Medellín**
- **Cartagena**
- **Barranquilla**
- **Cali**

#### Atributos generados:
- **Ciudad**: selección aleatoria entre las 5 ciudades.
- **Costo de ejecución**: entre 0.001 y 10 millones de pesos.
- **# de empleos generados**: entre 0.01 y 15 mil.
- **Latitud y longitud**: aleatoria dentro de un rango representativo por ciudad.

In [1]:
import time
import sys
import os

# ============================================================================
# Preparación del entorno y carga de datos
# ============================================================================
# Este script importa y genera instancias para el problema de la mochila
# utilizando una fuente externa ('InsKnapsack.py') ubicada en un directorio
# de instancias personalizado.
# ============================================================================

# --------------------------------------------------------------------------
# 1. AJUSTAR EL PATH PARA ACCEDER A INSTANCIAS PERSONALIZADAS
# --------------------------------------------------------------------------
# Agrega la ruta '../Instances' al path para importar desde allí
sys.path.append(os.path.abspath("../Instances"))

# --------------------------------------------------------------------------
# 2. IMPORTAR Y GENERAR LOS DATOS DE PRUEBA
# --------------------------------------------------------------------------
from InsKnapsack import generar_datos

# Generar un conjunto de'n' "obras" (ítems) con beneficios y pesos
obras = generar_datos(100)

# --------------------------------------------------------------------------
# 3. PREPARAR ESTRUCTURA PARA ALMACENAR RESULTADOS
# --------------------------------------------------------------------------
resultados = []  # lista donde se irán guardando salidas de algoritmos

# --------------------------------------------------------------------------
# 4. MOSTRAR DATOS GENERADOS (opcional)
# --------------------------------------------------------------------------
obras


,Ciudad,Costo de ejecución (en millones de pesos),# de empleos generados (en miles),Latitud,Longitud
0,Medellín,8.0,12.0,6.166654,-75.629120
1,Cartagena,7.0,9.0,10.371855,-75.561035
2,Cartagena,8.0,5.0,10.464352,-75.571365
3,Barranquilla,3.0,14.0,11.087343,-74.874611
4,Bogotá,3.0,10.0,4.728622,-74.139144
...,...,...,...,...,...
95,Cali,6.0,12.0,3.489379,-76.486129
96,Barranquilla,1.0,2.0,10.901539,-74.770359
97,Bogotá,2.0,9.0,4.564414,-74.090858
98,Cali,9.0,13.0,3.391680,-76.409873


## Optimización (MIP)

1. Conjuntos  
$O$: Conjunto de obras disponibles.

2. Parámetros  
$c_i$: Costo de ejecución (en millones de pesos) de la obra $i \in O$  
$e_i$: Empleos generados (en miles) por la realización de la obra $i \in O$  
$p$: Presupuesto total disponible (en millones de pesos) para ejecutar obras de infraestructura vial

3. Variables de decisión  
$$
x_i =
\begin{cases}
1, & \text{si la obra } i \in O \text{ se lleva a cabo} \\
0, & \text{de lo contrario (d.l.c.)}
\end{cases}
$$

4. Función objetivo  
$$
\text{Maximizar} \sum_{i \in O} e_i \cdot x_i
$$

5. Restricciones  
    (1) Presupuesto disponible:  
$$
\sum_{i \in O} c_i \cdot x_i \leq p
$$
    (2) Naturaleza de las variables:  
$$
x_i \in \{0, 1\}, \quad \forall i \in O
$$

    Donde (1) garantiza que no se exceda el presupuesto disponible y (2) establece la naturaleza de las variables.


In [2]:
# ============================================================================
# Parametrización del problema: selección de obras bajo restricción presupuestal
# ============================================================================
# Se extraen los parámetros relevantes desde el DataFrame 'obras' para
# construir los insumos del modelo:
#   • Conjunto de decisiones
#   • Costos (c)
#   • Beneficios (empleos generados, e)
#   • Información geográfica (lat, log)
#   • Presupuesto total disponible
# ============================================================================

# --------------------------------------------------------------------------
# 1. CONJUNTO DE ÍTEMS (obras disponibles)
# --------------------------------------------------------------------------
O = list(obras.index)  # Lista de identificadores de obras (índices del DataFrame)

# --------------------------------------------------------------------------
# 2. PARÁMETROS DE CADA OBRA
# --------------------------------------------------------------------------

# ▸ c[o] : costo de ejecutar la obra o (en millones de COP)
c = dict(obras['Costo de ejecución (en millones de pesos)'].squeeze())

# ▸ e[o] : empleos generados por la obra o (en miles de personas)
e = dict(obras['# de empleos generados (en miles)'].squeeze())

# ▸ lat[o], log[o] : coordenadas geográficas de la obra o
lat = dict(obras['Latitud'].squeeze())   # latitud para visualización
log = dict(obras['Longitud'].squeeze())  # longitud para visualización

# --------------------------------------------------------------------------
# 3. RESTRICCIÓN PRESUPUESTAL
# --------------------------------------------------------------------------
# Presupuesto disponible en millones de COP (valor flexible)
# Aquí se fija como el mínimo entre:
#    ▸ número de obras
#    ▸ 100 millones
p = min(len(O), 100)


In [3]:
import pulp as lp

def optimizacion_knapsack(O, c, e, p):
    """
    ============================================================================
    Optimización tipo mochila (knapsack binaria) para selección de obras públicas
    ─────────────────────────────────────────────────────────────────────────────
    Objetivo:
      • Seleccionar un subconjunto de obras que **maximice la generación de empleo**
        bajo un presupuesto máximo disponible.

    Entradas
      • O : list
            Conjunto de identificadores de obras (ítems disponibles)
      • c : dict[obra → costo]
            Costo de ejecutar cada obra (en millones de COP)
      • e : dict[obra → empleos]
            Empleos generados por cada obra (en miles)
      • p : float
            Presupuesto total disponible (en millones de COP)

    Salida
      • model               : objeto del modelo PuLP (útil para trazabilidad)
      • x                  : dict[obra → variable binaria] con solución óptima
    ============================================================================

    Descripción resumida
    ---------------------------------------------------------------------------
      ▸ Modelo binario con función objetivo lineal:
              max   ∑ e[i]·x[i]
              s.a.  ∑ c[i]·x[i] ≤ p
                     x[i] ∈ {0,1}
      ▸ Se resuelve usando el solver por defecto de PuLP (CBC)
    """

    # -------------------------------------------------------------------------
    # 1. DEFINICIÓN DEL MODELO DE OPTIMIZACIÓN
    # -------------------------------------------------------------------------
    model = lp.LpProblem("Obras", lp.LpMaximize)

    # -------------------------------------------------------------------------
    # 2. VARIABLES DE DECISIÓN: x[i] = 1 si se selecciona la obra i
    # -------------------------------------------------------------------------
    x = {i: lp.LpVariable(f'x_{i}', lowBound=0, cat=lp.LpBinary) for i in O}

    # -------------------------------------------------------------------------
    # 3. FUNCIÓN OBJETIVO: maximizar empleos generados
    # -------------------------------------------------------------------------
    model += lp.lpSum(e[i] * x[i] for i in O)

    # -------------------------------------------------------------------------
    # 4. RESTRICCIONES
    # -------------------------------------------------------------------------

    # ▸ Presupuesto total no debe exceder p
    model += lp.lpSum(c[i] * x[i] for i in O) <= p

    # ▸ Restricción de binariedad ya está impuesta al definir las variables

    # -------------------------------------------------------------------------
    # 5. RESOLVER EL MODELO (usando CBC Solver)
    # -------------------------------------------------------------------------
    solver = lp.getSolver('PULP_CBC_CMD', msg=False)
    model.solve(solver)

    # -------------------------------------------------------------------------
    # 6. REPORTE DEL ESTADO DE SOLUCIÓN
    # -------------------------------------------------------------------------
    print(f'El optimizador llegó a una solución: {lp.LpStatus[model.status]}.')

    return model, x


In [4]:
# ============================================================================
# Ejecución y evaluación del modelo LP (mochila binaria con PuLP)
# ============================================================================
# Se mide el tiempo de ejecución, se extraen resultados relevantes y se
# almacenan en una lista de reportes para análisis posterior.
# ============================================================================

# --------------------------------------------------------------------------
# 1. MEDIR TIEMPO DE EJECUCIÓN
# --------------------------------------------------------------------------
start = time.perf_counter()

# ▸ Ejecutar el modelo exacto
model, x = optimizacion_knapsack(O, c, e, p)

end = time.perf_counter()

# --------------------------------------------------------------------------
# 2. EXTRACCIÓN DE RESULTADOS DE LA SOLUCIÓN
# --------------------------------------------------------------------------

# ▸ Valor óptimo de la función objetivo (empleos generados)
obj_lp = round(lp.value(model.objective), 3)

# ▸ Lista de obras seleccionadas según x[i] = 1
obras_selec_lp = [i for i in O if x[i].varValue == 1]

# ▸ Presupuesto total utilizado (suma de los costos seleccionados)
presupuesto_usado_lp = round(sum(c[i] for i in obras_selec_lp), 2)

# --------------------------------------------------------------------------
# 3. REPORTE POR CONSOLA
# --------------------------------------------------------------------------
print(f'FO (valor total): {obj_lp}')
print(f'Cantidad de obras seleccionadas: {len(obras_selec_lp)}.')
print(f'Presupuesto usado: {presupuesto_usado_lp}/{p}')
print(f"Tiempo (s): {end - start:.8f}")

# --------------------------------------------------------------------------
# 4. ALMACENAR RESULTADOS EN LISTA DE REPORTES
# --------------------------------------------------------------------------
resultados.append({
    "Método"             : "LP",
    "Empleos generados"  : obj_lp,
    "Obras seleccionadas": len(obras_selec_lp),
    "Presupuesto usado"  : presupuesto_usado_lp,
    "Tiempo (s)"         : end - start
})


El optimizador llegó a una solución: Optimal.
FO (valor total): 382.0
Cantidad de obras seleccionadas: 36.
Presupuesto usado: 100.0/100
Tiempo (s): 0.15620500


## Programación Dinámica

### Teoría

#### 1. ¿Qué es la Programación Dinámica?

La Programación Dinámica (PD) es una familia de métodos usados para resolver procesos de decisión secuenciales. PD se aplica cuando se conoce completamente el modelo del entorno, es decir:

- Las probabilidades de transición: $P(s' \mid s, a)$
- Las recompensas esperadas: $R(s, a, s')$

El objetivo es encontrar una política que maximice la recompensa acumulada esperada. Esto se hace resolviendo recursivamente las **ecuaciones de Bellman**.

#### 2. Componentes

Un problema secuencial se modela como un **Proceso de Decisión de Markov (MDP)** con:

- $S$: conjunto de estados.
- $A$: conjunto de acciones.
- $P(s' \mid s, a)$: probabilidad de transitar a $s'$ desde $s$ al tomar $a$.
- $R(s, a, s')$: recompensa esperada al ir de $s$ a $s'$ con acción $a$.
- $\gamma$: factor de descuento $(0 \leq \gamma \leq 1)$ que pondera recompensas futuras.

#### 3. Funciones de Valor

$v_\pi(s)$ es la recompensa esperada al seguir la política $\pi$ desde el estado $s$:

$$
v_\pi(s) = \mathbb{E}_\pi \left[ \sum_{t=0}^\infty \gamma^t R_{t+1} \mid S_0 = s \right]
$$

Ecuación de Bellman para $v_\pi$:

$$
v_\pi(s) = \sum_a \pi(a \mid s) \sum_{s'} P(s' \mid s, a) \left[ R(s, a, s') + \gamma v_\pi(s') \right]
$$

#### 4. Objetivo: encontrar la política óptima

Queremos encontrar una política $\pi^*$ tal que:

$$
v_{\pi^*}(s) \geq v_\pi(s), \quad \text{para todo } s \text{ y para toda } \pi
$$

La función de valor óptima $v_*$ satisface la **ecuación de Bellman óptima**:

$$
v_*(s) = \max_a \sum_{s'} P(s' \mid s, a) \left[ R(s, a, s') + \gamma v_*(s') \right]
$$

### Algoritmos de Solución

| **Símbolo** | **Definición (problema _knapsack_)** | **Comentarios** |
|-------------|--------------------------------------|-----------------|
| **Estados $\mathcal{S}$** | Par ordenado $s=(i,c)$ donde:<br>• $i \in \{0,\dots,n\}$ es el índice del **próximo** objeto por decidir.<br>• $c \in \{0,\dots,W\}$ es la **capacidad restante**. | Esta representación mantiene la información necesaria: cuántos ítems quedan por evaluar y cuánta capacidad nos queda. |
| **Acciones $\mathcal{A}(s)$** | Para $i < n$:<br>• **Tomar** el objeto $i$ (solo si $w_i \le c$).<br>• **Omitir** el objeto $i$.<br>Para $i = n$ no hay acciones (estado terminal). | Con “tomar” avanzamos al siguiente objeto y restamos su peso; con “omitir” solo avanzamos. |
| **Transición $p(s',r \mid s,a)$** | **Determinista**: al elegir acción $a$ desde $(i,c)$:<br>– Si **tomar**: $s' = (i+1,\,c-w_i)$, recompensa $r = v_i$.<br>– Si **omitir**: $s' = (i+1,\,c)$, recompensa $r = 0$. | No hay azar; todos los caminos están completamente determinados. |
| **Recompensa $\mathcal{R}(s,a)$** | Valor inmediato del objeto si se toma; $0$ si se omite. | La **suma total de recompensas** a lo largo del episodio es exactamente el valor acumulado de la mochila. |
| **Factor de descuento $\gamma$** | $\gamma = 1$ | El episodio es finito (máx. $n$ decisiones) y no nos preocupa preferir valor temprano o tardío. |
| **Distribución inicial $p_0(s)$** | Único estado inicial $s_0 = (0,W)$. | Al empezar, contamos con cero objetos seleccionados y la totalidad de la capacidad. |
| **Estados terminales $\mathcal{T}$** | Todos los estados con $i = n$. | Una vez evaluados los $n$ objetos ya no hay más decisiones; el episodio termina. |

In [5]:
# ============================================================================
# Importación de módulos del proyecto: entorno, algoritmos y visualización
# ============================================================================
# Se agregan las rutas correspondientes al entorno del problema de mochila,
# las implementaciones de algoritmos de optimización dinámica y funciones
# de visualización asociadas.
# ============================================================================

# --------------------------------------------------------------------------
# 1. AJUSTAR RUTAS AL PROYECTO (para imports relativos personalizados)
# --------------------------------------------------------------------------
sys.path.append(os.path.abspath("../DP/Env"))         # entorno del problema
sys.path.append(os.path.abspath("../DP/Algorithms/")) # algoritmos RL / DP
sys.path.append(os.path.abspath("../DP/Visual/"))     # visualización de políticas / valores

# --------------------------------------------------------------------------
# 2. IMPORTAR COMPONENTES DEL ENTORNO
# --------------------------------------------------------------------------
from Knapsack import KnapsackEnv  # Clase que define el entorno tipo mochila

# --------------------------------------------------------------------------
# 3. IMPORTAR ALGORITMOS DE CONTROL / VALORACIÓN
# --------------------------------------------------------------------------
from value_iteration import value_iteration           # iteración de valores
from policy_evaluation import policy_evaluation       # evaluación de políticas
from policy_iteration import policy_iteration         # iteración de políticas

# --------------------------------------------------------------------------
# 4. IMPORTAR FUNCIONES DE VISUALIZACIÓN
# --------------------------------------------------------------------------
from value_states import value_states_visual          # visualización de V(s)
from policy_dag import draw_policy_dag                # grafo de política óptima


#### Datos

In [6]:
# ============================================================================
# Creación del entorno KnapsackEnv para RL/DP
# ============================================================================
# Se instancia el entorno tipo mochila con pesos, valores y capacidad a partir
# de los datos originales de obras. Este entorno será usado por los algoritmos
# de iteración de valores y políticas.
# ============================================================================

# --------------------------------------------------------------------------
# 1. PARAMETRIZAR EL ENTORNO
# --------------------------------------------------------------------------

weights  = list(c.values())  # pesos de los ítems: costos (millones de COP)
values   = list(e.values())  # beneficios de los ítems: empleos (miles)
capacity = p                 # capacidad total: presupuesto disponible

# --------------------------------------------------------------------------
# 2. CREAR INSTANCIA DEL ENTORNO
# --------------------------------------------------------------------------
env = KnapsackEnv(weights, values, capacity)

# --------------------------------------------------------------------------
# 3. MOSTRAR INFORMACIÓN DEL ENTORNO
# --------------------------------------------------------------------------
print(env)  # resumen textual del entorno (definido por __str__ o __repr__)


KnapsackEnv(#_Objetos = 100, Capacidad = 100, #_Estados = 10201)


#### Policy Evaluation

Input $\pi$, la política a evaluar  
Inicializar un arreglo $V(s)=0$, para todo $s\in\mathcal{S}^+$  

**Repetir**  
&nbsp;&nbsp;&nbsp;&nbsp;$\Delta \leftarrow 0$  
&nbsp;&nbsp;&nbsp;&nbsp;**Para cada** $s\in\mathcal{S}$:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$v \leftarrow V(s)$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$V(s) \leftarrow \displaystyle\sum_a \pi(a\mid s)\sum_{s',\,r} p(s',r \mid s,a)\,[\,r + \gamma\,V(s')\,]$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\Delta \leftarrow \max\!\bigl(\Delta,\;\lvert v - V(s)\rvert\bigr)$  

**Hasta que** $\Delta < \theta$ (un número pequeño y positivo)  

**Salida** $V \approx v_\pi$

In [7]:
# ============================================================================
# Evaluación de una política heurística (greedy con 'take' si es legal)
# ============================================================================
# Se define una política inicial que toma cada ítem si es legal hacerlo.
# Luego, se evalúa su valor esperado (V) en todo el espacio de estados,
# y se reportan métricas similares al enfoque LP.
# ============================================================================

# --------------------------------------------------------------------------
# 1. DEFINIR POLÍTICA PROPUESTA: tomar si es posible
# --------------------------------------------------------------------------
# Cada estado s = (i, capacidad restante)
# Acción: 'take' si está permitida; si no, 'skip'
propose_policy = {
    s: ('take' if 'take' in env.actions(s) else 'skip')
    for s in env.state_space()
}

policy = propose_policy.copy()

# --------------------------------------------------------------------------
# 2. EVALUAR LA POLÍTICA (valor esperado en cada estado)
# --------------------------------------------------------------------------
t0 = time.perf_counter()

# ▸ Evaluación de la política determinista 'propose_policy'
V = policy_evaluation(env, propose_policy)

elapsed = time.perf_counter() - t0
print(f"\nTiempo de ejecución: {elapsed:.6f} s")

# --------------------------------------------------------------------------
# 3. REPORTAR RESULTADOS DESDE LA POLÍTICA EVALUADA
# --------------------------------------------------------------------------
# Método auxiliar del entorno: devuelve
#  - valor total (beneficio alcanzado)
#  - cantidad de objetos tomados
#  - peso total utilizado
valor_total, objetos_tomados, peso_total = env.report_from_policy(policy)

# --------------------------------------------------------------------------
# 4. ALMACENAR RESULTADOS EN LISTA DE REPORTES
# --------------------------------------------------------------------------
resultados.append({
    "Método"             : "Política evaluation",
    "Empleos generados"  : round(valor_total, 3),
    "Obras seleccionadas": len(objetos_tomados),
    "Presupuesto usado"  : round(peso_total, 2),
    "Tiempo (s)"         : elapsed
})



Tiempo de ejecución: 0.340723 s
Objetos seleccionados:
  • Obj  0: peso=8.0, valor=12.0
  • Obj  1: peso=7.0, valor=9.0
  • Obj  2: peso=8.0, valor=5.0
  • Obj  3: peso=3.0, valor=14.0
  • Obj  4: peso=3.0, valor=10.0
  • Obj  5: peso=5.0, valor=12.0
  • Obj  6: peso=1.0, valor=13.0
  • Obj  7: peso=5.0, valor=10.0
  • Obj  8: peso=4.0, valor=15.0
  • Obj  9: peso=5.0, valor=11.0
  • Obj 10: peso=7.0, valor=10.0
  • Obj 11: peso=3.0, valor=2.0
  • Obj 12: peso=8.0, valor=13.0
  • Obj 13: peso=5.0, valor=6.0
  • Obj 14: peso=2.0, valor=11.0
  • Obj 15: peso=5.0, valor=11.0
  • Obj 16: peso=9.0, valor=13.0
  • Obj 17: peso=7.0, valor=13.0
  • Obj 19: peso=5.0, valor=8.0
FO (valor total):    198.0
Presupuesto usado:   100.0/100


#### Policy Iteration

1. **Inicialización**  
   $V(s)\in\mathbb{R}$ y $\pi(s)\in\mathcal{A}(s)$ arbitrarios, $\forall\,s\in\mathcal{S}$  

2. **Evaluación de la política**  
   **Repetir**  
   &nbsp;&nbsp;$\Delta \leftarrow 0$  
   &nbsp;&nbsp;**Para cada** $s\in\mathcal{S}$:  
   &nbsp;&nbsp;&nbsp;&nbsp;$v \leftarrow V(s)$  
   &nbsp;&nbsp;&nbsp;&nbsp;$V(s) \leftarrow \displaystyle\sum_{s',\,r} p(s',r \mid s,\pi(s))\,[\,r + \gamma\,V(s')\,]$  
   &nbsp;&nbsp;&nbsp;&nbsp;$\Delta \leftarrow \max\!\bigl(\Delta,\;|v - V(s)|\bigr)$  
   **Hasta que** $\Delta < \theta$ (un número pequeño y positivo)  

3. **Mejora de la política**  
   $policy\text{-}stable \leftarrow \text{true}$  
   **Para cada** $s\in\mathcal{S}$:  
   &nbsp;&nbsp;$a \leftarrow \pi(s)$  
   &nbsp;&nbsp;$\displaystyle\pi(s) \leftarrow \arg\max_a \sum_{s',\,r} p(s',r \mid s,a)\,[\,r + \gamma\,V(s')\,]$  
   &nbsp;&nbsp;**Si** $a \ne \pi(s)$ **entonces** $policy\text{-}stable \leftarrow \text{false}$  
   **Si** $policy\text{-}stable$ **entonces** detener y devolver $V$ y $\pi$; **de lo contrario** volver al paso 2

In [8]:
# ============================================================================
# Aplicación del algoritmo de Iteración de Políticas (policy_iteration)
# ============================================================================
# Se parte de una política inicial (`policy`) y se itera entre evaluación y mejora
# hasta alcanzar una política óptima. Luego se mide su rendimiento.
# ============================================================================

# --------------------------------------------------------------------------
# 1. EJECUTAR POLICY ITERATION
# --------------------------------------------------------------------------
t0 = time.perf_counter()

# ▸ Aplicar iteración de políticas sobre el entorno de mochila
star_policy, V_star = policy_iteration(env, policy)

elapsed = time.perf_counter() - t0
print(f"\nTiempo de ejecución: {elapsed:.6f} s")

# --------------------------------------------------------------------------
# 2. EVALUAR LA POLÍTICA ÓPTIMA OBTENIDA
# --------------------------------------------------------------------------
valor_total, objetos_tomados, peso_total = env.report_from_policy(star_policy)

# --------------------------------------------------------------------------
# 3. ALMACENAR RESULTADOS EN LISTA DE REPORTE
# --------------------------------------------------------------------------
resultados.append({
    "Método"             : "Policy iteration",
    "Empleos generados"  : round(valor_total, 3),
    "Obras seleccionadas": len(objetos_tomados),
    "Presupuesto usado"  : round(peso_total, 2),
    "Tiempo (s)"         : elapsed
})



Tiempo de ejecución: 8.264731 s
Objetos seleccionados:
  • Obj  3: peso=3.0, valor=14.0
  • Obj  4: peso=3.0, valor=10.0
  • Obj  5: peso=5.0, valor=12.0
  • Obj  6: peso=1.0, valor=13.0
  • Obj  8: peso=4.0, valor=15.0
  • Obj 14: peso=2.0, valor=11.0
  • Obj 15: peso=5.0, valor=11.0
  • Obj 25: peso=2.0, valor=6.0
  • Obj 26: peso=4.0, valor=10.0
  • Obj 27: peso=2.0, valor=10.0
  • Obj 28: peso=2.0, valor=13.0
  • Obj 29: peso=2.0, valor=12.0
  • Obj 34: peso=1.0, valor=13.0
  • Obj 40: peso=3.0, valor=8.0
  • Obj 42: peso=3.0, valor=11.0
  • Obj 44: peso=6.0, valor=14.0
  • Obj 48: peso=1.0, valor=6.0
  • Obj 49: peso=4.0, valor=14.0
  • Obj 50: peso=4.0, valor=11.0
  • Obj 51: peso=1.0, valor=9.0
  • Obj 53: peso=3.0, valor=10.0
  • Obj 56: peso=4.0, valor=12.0
  • Obj 59: peso=3.0, valor=13.0
  • Obj 61: peso=3.0, valor=8.0
  • Obj 68: peso=2.0, valor=13.0
  • Obj 72: peso=2.0, valor=5.0
  • Obj 74: peso=3.0, valor=12.0
  • Obj 76: peso=4.0, valor=11.0
  • Obj 86: peso=1.0, valo

#### Value Iteration

Inicializar el arreglo $V$ de forma arbitraria (p. ej. $V(s)=0$ para todo $s\in\mathcal{S}^+$)  

**Repetir**  
&nbsp;&nbsp;$\Delta \leftarrow 0$  
&nbsp;&nbsp;**Para cada** $s\in\mathcal{S}$:  
&nbsp;&nbsp;&nbsp;&nbsp;$v \leftarrow V(s)$  
&nbsp;&nbsp;&nbsp;&nbsp;$\displaystyle V(s) \leftarrow \max_a \sum_{s',\,r} p(s',r \mid s,a)\,[\,r + \gamma\,V(s')\,]$  
&nbsp;&nbsp;&nbsp;&nbsp;$\Delta \leftarrow \max\!\bigl(\Delta,\;|\,v - V(s)\,|\bigr)$  
**Hasta que** $\Delta < \theta$ (un número pequeño y positivo)  

**Salida** una política determinista $\pi$ tal que  
$\displaystyle \pi(s) = \arg\max_a \sum_{s',\,r} p(s',r \mid s,a)\,[\,r + \gamma\,V(s')\,]$

In [9]:
# ============================================================================
# Aplicación del algoritmo de Iteración de Valores (value_iteration)
# ============================================================================
# Calcula V* mediante actualización iterativa de valores de estado, luego
# deriva una política óptima π* de forma greedy respecto a V*.
# ============================================================================

# --------------------------------------------------------------------------
# 1. EJECUTAR VALUE ITERATION
# --------------------------------------------------------------------------
t0 = time.perf_counter()

# ▸ Ejecutar algoritmo de iteración de valores sobre el entorno
opt_policy, V_opt = value_iteration(env)

elapsed = time.perf_counter() - t0
print(f"\nTiempo de ejecución: {elapsed:.6f} s")

# --------------------------------------------------------------------------
# 2. EVALUAR LA POLÍTICA DERIVADA DE V*
# --------------------------------------------------------------------------
valor_total, objetos_tomados, peso_total = env.report_from_policy(opt_policy)

# --------------------------------------------------------------------------
# 3. ALMACENAR RESULTADOS EN LISTA DE REPORTE
# --------------------------------------------------------------------------
resultados.append({
    "Método"             : "Value iteration",
    "Empleos generados"  : round(valor_total, 3),
    "Obras seleccionadas": len(objetos_tomados),
    "Presupuesto usado"  : round(peso_total, 2),
    "Tiempo (s)"         : elapsed
})



Tiempo de ejecución: 1.330796 s
Objetos seleccionados:
  • Obj  3: peso=3.0, valor=14.0
  • Obj  4: peso=3.0, valor=10.0
  • Obj  5: peso=5.0, valor=12.0
  • Obj  6: peso=1.0, valor=13.0
  • Obj  8: peso=4.0, valor=15.0
  • Obj 14: peso=2.0, valor=11.0
  • Obj 15: peso=5.0, valor=11.0
  • Obj 25: peso=2.0, valor=6.0
  • Obj 26: peso=4.0, valor=10.0
  • Obj 27: peso=2.0, valor=10.0
  • Obj 28: peso=2.0, valor=13.0
  • Obj 29: peso=2.0, valor=12.0
  • Obj 34: peso=1.0, valor=13.0
  • Obj 40: peso=3.0, valor=8.0
  • Obj 42: peso=3.0, valor=11.0
  • Obj 44: peso=6.0, valor=14.0
  • Obj 48: peso=1.0, valor=6.0
  • Obj 49: peso=4.0, valor=14.0
  • Obj 50: peso=4.0, valor=11.0
  • Obj 51: peso=1.0, valor=9.0
  • Obj 53: peso=3.0, valor=10.0
  • Obj 56: peso=4.0, valor=12.0
  • Obj 59: peso=3.0, valor=13.0
  • Obj 61: peso=3.0, valor=8.0
  • Obj 68: peso=2.0, valor=13.0
  • Obj 72: peso=2.0, valor=5.0
  • Obj 74: peso=3.0, valor=12.0
  • Obj 76: peso=4.0, valor=11.0
  • Obj 86: peso=1.0, valo

### Visualización

#### Value Functions

In [10]:
# ============================================================================
# Visualización condicional de V(s) para cada método aplicado
# ============================================================================
# Se muestran mapas de calor solo si el número de ítems (env.n) es razonable.
# Esto evita congestión visual en instancias grandes.
# ============================================================================

if env.n <= 20:

    # ----------------------------------------------------------------------
    # Evaluación de política heurística
    # ----------------------------------------------------------------------
    print("Policy Evaluation")
    value_states_visual(env, V)
    print()

    # ----------------------------------------------------------------------
    # Política óptima vía iteración de políticas
    # ----------------------------------------------------------------------
    print("Policy Iteration")
    value_states_visual(env, V_star)
    print()

    # ----------------------------------------------------------------------
    # Política óptima vía iteración de valores
    # ----------------------------------------------------------------------
    print("Value Iteration")
    value_states_visual(env, V_opt)
    print()

else:
    print("Para visualizar, use una instancia más pequeña.")


Para visualizar, use una instancia más pequeña.


#### Policy

In [11]:
# ============================================================================
# Visualización condicional del DAG de decisiones para cada política evaluada
# ============================================================================
# Dibuja el grafo dirigido de estados con la trayectoria tomada por:
#   • Política heurística
#   • Iteración de políticas
#   • Iteración de valores
# Solo se activa para instancias pequeñas (n ≤ 20).
# ============================================================================

if env.n <= 20:

    # ----------------------------------------------------------------------
    # Política heurística: tomar cuando sea legal
    # ----------------------------------------------------------------------
    print("Policy Evaluation")
    draw_policy_dag(env, propose_policy, initial_state=(0, env.capacity))
    print()

    # ----------------------------------------------------------------------
    # Política óptima vía iteración de políticas
    # ----------------------------------------------------------------------
    print("Policy Iteration")
    draw_policy_dag(env, star_policy, initial_state=(0, env.capacity))
    print()

    # ----------------------------------------------------------------------
    # Política óptima vía iteración de valores
    # ----------------------------------------------------------------------
    print("Value Iteration")
    draw_policy_dag(env, opt_policy, initial_state=(0, env.capacity))
    print()

else:
    print("Para visualizar, use una instancia más pequeña.")


Para visualizar, use una instancia más pequeña.


## Heurísticas

### Greedy

Esta función aplica un algoritmo **greedy (voraz)** que selecciona obras públicas ordenadas por la mayor cantidad de empleos generados, siempre que no excedan el presupuesto disponible.

In [12]:
def greedy_knapsack(O, c, e, p, criterio):
    """
    ============================================================================
    Algoritmo voraz (greedy) para selección de obras públicas bajo presupuesto
    ─────────────────────────────────────────────────────────────────────────────
    Selecciona obras en orden decreciente según un criterio dado, y las incluye
    mientras no se supere el presupuesto total.

    Esta aproximación no garantiza optimalidad, pero es eficiente y útil como base.

    Entradas
      • O : list
              Conjunto de identificadores de obras
      • c : dict[obra → costo]
              Costo de cada obra (en millones de COP)
      • e : dict[obra → empleos]
              Empleos generados por cada obra (en miles)
      • p : float
              Presupuesto total disponible
      • criterio : dict[obra → float]
              Métrica para priorizar la selección (ej. e[i]/c[i], e[i], etc.)

    Salida
      • obj_greedy               : empleos generados (miles)
      • obras_selec_greedy      : lista de obras seleccionadas
      • presupuesto_usado_greedy: millones de COP utilizados
    ============================================================================
    """

    # -------------------------------------------------------------------------
    # 1. INICIALIZACIÓN
    # -------------------------------------------------------------------------
    presupuesto_actual = 0.0
    obras_selec_greedy = []

    # -------------------------------------------------------------------------
    # 2. ORDENAR OBRAS SEGÚN EL CRITERIO ESPECIFICADO
    # -------------------------------------------------------------------------
    obras_ordenadas = sorted(O, key=lambda i: criterio[i], reverse=True)

    # -------------------------------------------------------------------------
    # 3. SELECCIÓN VORAZ: agregar mientras no se supere el presupuesto
    # -------------------------------------------------------------------------
    for i in obras_ordenadas:
        if presupuesto_actual + c[i] <= p:
            obras_selec_greedy.append(i)
            presupuesto_actual += c[i]

    # -------------------------------------------------------------------------
    # 4. RESULTADOS: empleos generados y presupuesto utilizado
    # -------------------------------------------------------------------------
    obj_greedy = round(sum(e[i] for i in obras_selec_greedy), 2)
    presupuesto_usado_greedy = round(presupuesto_actual, 2)

    return obj_greedy, obras_selec_greedy, presupuesto_usado_greedy


In [13]:
# ============================================================================
# Ejecución del algoritmo greedy (voraz) bajo criterio de impacto absoluto (e)
# ============================================================================
# Se priorizan las obras que generan más empleos en términos absolutos.
# Se mide tiempo de ejecución y se registran los resultados.
# ============================================================================

# --------------------------------------------------------------------------
# 1. MEDIR TIEMPO DE EJECUCIÓN
# --------------------------------------------------------------------------
start = time.perf_counter()

# ▸ Criterio: empleos absolutos
criterio = e

# ▸ Ejecutar algoritmo voraz
obj_greedy, obras_selec_greedy, presupuesto_usado_greedy = greedy_knapsack(O, c, e, p, criterio)

end = time.perf_counter()

# ▸ Reporte por consola
print(f"Tiempo (s): {end - start:.8f}")
print(f'La cantidad de empleos generados es de: {obj_greedy} (en miles).')
print(f'La cantidad de obras seleccionadas fue de: {len(obras_selec_greedy)}.')
print(f'El presupuesto usado fue (COP Millones): {presupuesto_usado_greedy}/{p}')

# --------------------------------------------------------------------------
# 2. ALMACENAR RESULTADOS EN LISTA DE REPORTE
# --------------------------------------------------------------------------
resultados.append({
    "Método"             : "Greedy (e)",
    "Empleos generados"  : obj_greedy,
    "Obras seleccionadas": len(obras_selec_greedy),
    "Presupuesto usado"  : presupuesto_usado_greedy,
    "Tiempo (s)"         : end - start
})


Tiempo (s): 0.00008488
La cantidad de empleos generados es de: 269.0 (en miles).
La cantidad de obras seleccionadas fue de: 20.
El presupuesto usado fue (COP Millones): 100.0/100


In [14]:
# ============================================================================
# Ejecución del algoritmo greedy (voraz) bajo criterio eficiencia e/c
# ============================================================================
# Se priorizan las obras con mayor impacto por unidad de costo.
# Se mide tiempo de ejecución y se registran los resultados.
# ============================================================================

# --------------------------------------------------------------------------
# 1. MEDIR TIEMPO DE EJECUCIÓN
# --------------------------------------------------------------------------
start = time.perf_counter()

# ▸ Definir criterio: eficiencia = empleos por unidad de costo
criterio = {i: e[i] / c[i] for i in O}

# ▸ Ejecutar algoritmo voraz
obj_greedy, obras_selec_greedy, presupuesto_usado_greedy = greedy_knapsack(O, c, e, p, criterio)

end = time.perf_counter()

# ▸ Reporte por consola
print(f"Tiempo (s): {end - start:.8f}")
print(f'La cantidad de empleos generados es de: {obj_greedy} (en miles).')
print(f'La cantidad de obras seleccionadas fue de: {len(obras_selec_greedy)}.')
print(f'El presupuesto usado fue (COP Millones): {presupuesto_usado_greedy}/{p}')

# --------------------------------------------------------------------------
# 2. ALMACENAR RESULTADOS EN LISTA DE REPORTE
# --------------------------------------------------------------------------
resultados.append({
    "Método"             : "Greedy (e/c)",
    "Empleos generados"  : obj_greedy,
    "Obras seleccionadas": len(obras_selec_greedy),
    "Presupuesto usado"  : presupuesto_usado_greedy,
    "Tiempo (s)"         : end - start
})


Tiempo (s): 0.00019575
La cantidad de empleos generados es de: 381.0 (en miles).
La cantidad de obras seleccionadas fue de: 37.
El presupuesto usado fue (COP Millones): 100.0/100


### Random

Esta función implementa una estrategia aleatoria para seleccionar obras públicas bajo un presupuesto dado. Las obras se mezclan aleatoriamente y se seleccionan en ese orden si hay presupuesto disponible.

In [15]:
import random

def random_knapsack(O, c, e, p):
    """
    ============================================================================
    Heurística aleatoria para selección de obras bajo restricción presupuestal
    ─────────────────────────────────────────────────────────────────────────────
    Selecciona obras en orden aleatorio (sin reemplazo) mientras no se supere
    el presupuesto total disponible.

    Esta estrategia no garantiza optimalidad, pero sirve como línea base rápida
    para comparar con algoritmos deterministas.

    Entradas
      • O : list
              Identificadores de obras disponibles
      • c : dict[obra → costo]
              Costo por obra (en millones de COP)
      • e : dict[obra → empleos]
              Empleos generados por cada obra (en miles)
      • p : float
              Presupuesto total disponible
      • report : bool
              Si es True, imprime trazas con resultados

    Salidas
      • obj_rand               : empleos generados (miles)
      • obras_selec_rand      : lista de obras seleccionadas
      • presupuesto_usado_rand: millones de COP utilizados
    ============================================================================
    """

    # -------------------------------------------------------------------------
    # 1. INICIALIZACIÓN
    # -------------------------------------------------------------------------
    presupuesto_actual = 0.0
    obras_selec_rand = []

    # -------------------------------------------------------------------------
    # 2. DESORDENAR OBRAS ALEATORIAMENTE (sin reemplazo)
    # -------------------------------------------------------------------------
    obras_random = random.sample(O, len(O))

    # -------------------------------------------------------------------------
    # 3. SELECCIÓN: agregar obras mientras quepa en el presupuesto
    # -------------------------------------------------------------------------
    for i in obras_random:
        if presupuesto_actual + c[i] <= p:
            obras_selec_rand.append(i)
            presupuesto_actual += c[i]

    # -------------------------------------------------------------------------
    # 4. CÁLCULO DE RESULTADOS
    # -------------------------------------------------------------------------
    obj_rand = round(sum(e[i] for i in obras_selec_rand), 2)
    presupuesto_usado_rand = round(presupuesto_actual, 2)

    return obj_rand, obras_selec_rand, presupuesto_usado_rand


In [16]:
# ============================================================================
# Búsqueda aleatoria intensiva (Random Search) con múltiples repeticiones
# ============================================================================
# Ejecuta múltiples soluciones aleatorias (random_knapsack) y conserva la mejor.
# Esto permite observar el mejor rendimiento posible de una estrategia estocástica.
# ============================================================================

# --------------------------------------------------------------------------
# 1. CONFIGURACIÓN DEL EXPERIMENTO
# --------------------------------------------------------------------------
n_iteraciones      = 1000                    # número total de repeticiones
mejor_obj          = -1                      # valor objetivo máximo observado
mejor_obras        = None                    # lista de obras correspondientes
mejor_presupuesto  = 0.0
mejor_tiempo       = 0.0

start_global = time.perf_counter()           # tiempo total del experimento

# --------------------------------------------------------------------------
# 2. EJECUTAR REPETIDAMENTE LA HEURÍSTICA ALEATORIA
# --------------------------------------------------------------------------
for _ in range(n_iteraciones):
    start = time.perf_counter()

    obj_rand, obras_selec_rand, presupuesto_usado_rand = random_knapsack(O, c, e, p)

    end = time.perf_counter()

    # ▸ Actualizar la mejor solución si se mejora la función objetivo
    if obj_rand > mejor_obj:
        mejor_obj         = obj_rand
        mejor_obras       = obras_selec_rand
        mejor_presupuesto = presupuesto_usado_rand
        mejor_tiempo      = end - start

end_global = time.perf_counter()

# --------------------------------------------------------------------------
# 3. REPORTE POR CONSOLA
# --------------------------------------------------------------------------
print(f'La cantidad de empleos generados es de: {mejor_obj} (en miles).')
print(f'La cantidad de obras seleccionadas fue de: {len(mejor_obras)}.')
print(f'El presupuesto usado fue (COP Millones): {mejor_presupuesto}/{p}')
print(f"Tiempo mejor sol (s): {mejor_tiempo:.8f}")
print(f"Tiempo total (s): {end_global:.8f}")

# --------------------------------------------------------------------------
# 4. REGISTRAR EN LA LISTA DE RESULTADOS
# --------------------------------------------------------------------------
resultados.append({
    "Método"             : f"Random (best of {n_iteraciones})",
    "Empleos generados"  : mejor_obj,
    "Obras seleccionadas": len(mejor_obras),
    "Presupuesto usado"  : mejor_presupuesto,
    "Tiempo (s)"         : end_global
})


La cantidad de empleos generados es de: 242.0 (en miles).
La cantidad de obras seleccionadas fue de: 24.
El presupuesto usado fue (COP Millones): 100.0/100
Tiempo mejor sol (s): 0.00001896
Tiempo total (s): 12376.27045071


### Búsqueda Local con Intercambios (Swaps)

Esta función mejora una solución inicial del problema de selección de obras mediante una estrategia de **búsqueda local**. En cada iteración:

- Se elimina una obra seleccionada.
- Se prueba reemplazarla por una obra no seleccionada.
- Si el intercambio **no viola el presupuesto** y **mejora el número de empleos**, se acepta.

In [17]:
def swaps_knapsack(O, c, e, p, sol_inicial, n=100, report=False):
    """
    ============================================================================
    Búsqueda local por swaps para mejora de soluciones en problema de mochila
    ─────────────────────────────────────────────────────────────────────────────
    Parte de una solución inicial válida y aplica intercambios aleatorios entre
    obras seleccionadas y no seleccionadas, conservando presupuesto y buscando
    mejorar el total de empleos generados.

    Entradas
      • O           : lista de identificadores de todas las obras
      • c           : dict[obra → costo] en millones de COP
      • e           : dict[obra → empleos] generados en miles
      • p           : presupuesto máximo permitido (en millones)
      • sol_inicial : lista de obras seleccionadas (solución válida inicial)
      • n           : número de intentos de mejora (default = 100)
      • report      : si True, imprime detalles de mejoras aceptadas

    Salidas
      • mejor_obj        : total de empleos generados (miles)
      • mejor_sol        : lista de obras seleccionadas en la mejor solución
      • presupuesto_usado: presupuesto total utilizado (millones)
    ============================================================================
    """

    # -------------------------------------------------------------------------
    # 1. INICIALIZAR CON LA SOLUCIÓN INICIAL DADA
    # -------------------------------------------------------------------------
    mejor_sol = sol_inicial.copy()
    mejor_obj = sum(e[i] for i in mejor_sol)

    # -------------------------------------------------------------------------
    # 2. INTENTAR MEJORAR LA SOLUCIÓN MEDIANTE SWAPS ALEATORIOS
    # -------------------------------------------------------------------------
    for inter in range(1, n + 1):

        # ▸ Seleccionar una obra para quitar
        i = random.choice(mejor_sol)

        # ▸ Seleccionar una obra que no esté en la solución
        candidatos = list(set(O) - set(mejor_sol))
        j = random.choice(candidatos)

        # ▸ Proponer nueva solución: swap i → j
        nueva_sol = mejor_sol.copy()
        nueva_sol.remove(i)
        nueva_sol.append(j)

        # ▸ Calcular el costo total de la nueva solución
        costo = sum(c[k] for k in nueva_sol)

        # ▸ Verificar si respeta el presupuesto
        if costo <= p:

            # ▸ Calcular empleos generados por la nueva solución
            obj = sum(e[k] for k in nueva_sol)

            # ▸ Aceptar mejora si aumenta empleos
            if obj > mejor_obj:
                if report:
                    print(f'[Intento {inter}] Mejora aleatoria:')
                    print(f'  - Swap: {i} ➜ {j} (e: {e[i]} ➜ {e[j]}, c: {c[i]} ➜ {c[j]})')
                    print(f'  - Objetivo nuevo: {round(obj, 2)}')
                    print(f'  - Presupuesto: {round(costo, 2)}/{p}\n')

                mejor_sol, mejor_obj = nueva_sol, obj

    # -------------------------------------------------------------------------
    # 3. RESULTADO FINAL: REPORTAR MEJOR SOLUCIÓN ENCONTRADA
    # -------------------------------------------------------------------------
    presupuesto_usado = sum(c[i] for i in mejor_sol)

    print(f'Empleos generados: {round(mejor_obj, 2)} mil')
    print(f'Obras seleccionadas: {len(mejor_sol)}')
    print(f'Presupuesto usado: {round(presupuesto_usado, 2)}/{p}')

    return mejor_obj, mejor_sol, presupuesto_usado

In [18]:
# ============================================================================
# Mejora de la mejor solución aleatoria mediante búsqueda local por swaps
# ============================================================================
# Se parte de la mejor solución aleatoria (obras_selec_rand) y se intenta
# mejorar su desempeño aplicando intercambios aleatorios válidos.
# ============================================================================

# --------------------------------------------------------------------------
# 1. DEFINIR SOLUCIÓN INICIAL (punto de partida)
# --------------------------------------------------------------------------
obj_rand, obras_selec_rand, presupuesto_usado_rand = random_knapsack(O, c, e, p)
sol_inicial = obras_selec_rand

# --------------------------------------------------------------------------
# 2. MEDIR TIEMPO DE EJECUCIÓN
# --------------------------------------------------------------------------
start = time.perf_counter()

# ▸ Ejecutar búsqueda local (hasta 10,000 intentos de mejora)
obj_rand_swaps, obras_selec_rand_swaps, presupuesto_usado_rand_swaps = swaps_knapsack(
    O, c, e, p, sol_inicial, n=10_000, report=False
)

end = time.perf_counter()

# ▸ Reporte por consola
print(f"Tiempo (s): {end - start:.8f}")

# --------------------------------------------------------------------------
# 3. REGISTRAR RESULTADOS EN LA LISTA DE MÉTODOS
# --------------------------------------------------------------------------
resultados.append({
    "Método"             : "Random + Swaps",
    "Empleos generados"  : obj_rand_swaps,
    "Obras seleccionadas": len(obras_selec_rand_swaps),
    "Presupuesto usado"  : presupuesto_usado_rand_swaps,
    "Tiempo (s)"         : end - start
})


Empleos generados: 257.0 mil
Obras seleccionadas: 19
Presupuesto usado: 97.0/100
Tiempo (s): 0.06797892


## Resultados

In [19]:
import pandas as pd

# -- Crear DataFrame desde la lista de resultados --
df_resultados = pd.DataFrame(resultados)

# -- Establecer 'Método' como índice para facilitar la comparación --
df_resultados.set_index("Método", inplace=True)

# -- Visualizar resultados --
df_resultados

,Empleos generados,Obras seleccionadas,Presupuesto usado,Tiempo (s)
Método,,,,
LP,382.0,36,100.0,0.156205
Política evaluation,198.0,19,100.0,0.340723
Policy iteration,382.0,36,100.0,8.264731
Value iteration,382.0,36,100.0,1.330796
Greedy (e),269.0,20,100.0,0.000085
Greedy (e/c),381.0,37,100.0,0.000196
Random (best of 1000),242.0,24,100.0,12376.270451
Random + Swaps,257.0,19,97.0,0.067979


In [20]:
# -- Tomar la solución base de referencia (LP) --
base_lp = df_resultados.loc["LP"]

# -- Calcular el GAP porcentual relativo al método LP (excepto para Tiempo) --
gap_df = ((df_resultados - base_lp) / base_lp) * 100

# -- Reemplazar la columna de tiempo con "x veces LP" en lugar de porcentaje --
tiempo_ratio = (df_resultados["Tiempo (s)"] / base_lp["Tiempo (s)"]).round(4)

# -- Redondear GAPs a 3 decimales --
gap_df = gap_df.round(3)

# -- Renombrar columnas GAP --
gap_df.columns = [col + " GAP (%)" for col in gap_df.columns]

# -- Reemplazar columna de tiempo con "x veces LP" --
gap_df["Tiempo (s) (x veces LP)"] = tiempo_ratio

# -- Eliminar columna anterior de GAP de tiempo --
gap_df = gap_df.drop(columns=["Tiempo (s) GAP (%)"])

# -- Mostrar tabla --
gap_df

,Empleos generados GAP (%),Obras seleccionadas GAP (%),Presupuesto usado GAP (%),Tiempo (s) (x veces LP)
Método,,,,
LP,0.000,0.000,0.0,1.0000
Política evaluation,-48.168,-47.222,0.0,2.1813
Policy iteration,0.000,0.000,0.0,52.9095
Value iteration,0.000,0.000,0.0,8.5195
Greedy (e),-29.581,-44.444,0.0,0.0005
Greedy (e/c),-0.262,2.778,0.0,0.0013
Random (best of 1000),-36.649,-33.333,0.0,79230.9494
Random + Swaps,-32.723,-47.222,-3.0,0.4352
